In [ ]:
from os import path

import xarray as xr
import numpy as np
import pandas as pd
from dask import array as da


from matplotlib import pyplot as plt

import panel as pn
import holoviews as hv
import geoviews as gv

from datetime import datetime as dt, UTC, timedelta
from pyxlma import coords


from cartopy import crs as ccrs
from cartopy import feature as cfeat

import cmweather


from GOES_temp_tracking import read_tobac_ds, add_goes_data_to_tobac_path

pn.extension() 
hv.extension('bokeh')

In [ ]:
test_x = np.arange(60, 80, 0.1)
test_y = np.arange(60, 80, 0.1)
fig2 = plt.figure()
fig = plt.figure()
ax = fig.gca()
handle = ax.scatter(test_x, test_y, c=test_y, cmap='BrBG')
fig.colorbar(handle, orientation='horizontal', label='Dew Point (F)')
fig

In [ ]:
ts = xr.open_dataset('/Volumes/LtgSSD/tobac_saves/tobac_Save_20220602/timeseries_data_melt4700.nc', chunks='auto')
ts.reindex(feature=tfm.feature.data, fill_value=np.nan).compute().sel(feature=6).feature_area

In [ ]:
rdr = xr.open_dataset('/Volumes/LtgSSD/nexrad_gridded/JUNE/20220602/KHGX20220602_000522_V06_grid.nc', chunks='auto')
rdr
radar_data = rdr
tpcs = coords.TangentPlaneCartesianSystem(radar_data.origin_latitude.data.compute()[0], radar_data.origin_longitude.data.compute()[0], radar_data.origin_altitude.data.compute()[0])
rcs = coords.RadarCoordinateSystem(radar_data.origin_latitude.data.compute()[0], radar_data.origin_longitude.data.compute()[0], radar_data.origin_altitude.data.compute()[0])
x3d, y3d, z3d = np.meshgrid(radar_data.x.data, radar_data.y.data, radar_data.z.data)
grid_X, grid_Y, grid_Z = tpcs.toECEF(x3d.flatten(), y3d.flatten(), z3d.flatten())
grid_r, grid_az, grid_el = rcs.fromECEF(grid_X, grid_Y, grid_Z)
grid_el = grid_el.reshape(x3d.shape)


In [ ]:
grid_el_closest_05 = grid_el > 0.5

In [ ]:
grid_el_closest_05 = np.argmax(grid_el_closest_05[:, :, :], axis=2)

In [ ]:
.compute()
fig = plt.figure()
ax = fig.gca()
pc = ax.pcolormesh(radar_data.x, radar_data.y, lowest_refl, cmap='ChaseSpectral', vmin=-10, vmax=80)
fig.colorbar(pc)

In [ ]:
rdr2 = xr.open_mfdataset('/Volumes/LtgSSD/nexrad_zarr/JUNE/20220602/*.zarr', engine='zarr', chunks='auto')
rdr2

In [ ]:
tfm = xr.open_dataset('/Volumes/LtgSSD/tobac_saves/tobac_Save_20220602/Track_features_merges.nc')


In [ ]:
tfm

In [ ]:
time_i_want = dt(2022, 6, 2, 18, 0, 0)


In [ ]:
tfm2 = xr.open_dataset('/Volumes/LtgSSD/tobac_saves/tobac_Save_20220602/Track_features_merges_augmented2.zarr', chunks='auto')
tfm2

In [ ]:
tfm2_filt = tfm2.where((tfm2.feature_time_index == 0).compute(), drop=True)

In [ ]:
tfm2.feature_time_index.data.compute()

In [ ]:
tfm2.feature_time

In [ ]:
min_sats_filt.time.astype(np.float64).data.flatten().shape

In [ ]:
min_sats_filt.feature.data.shape

In [ ]:
time_2d.T.shape

In [ ]:
min_sats_filt.data.shape

In [ ]:
tfm_time = tfm.sel(time=rdr.time.data[0], method='nearest')
tfm_time_features = np.unique(tfm_time.segmentation_mask)
tfm_time_features
for feat in tfm_time_features:
    if feat == 0:
        continue
    locations_of_feature = np.where(tfm_time.segmentation_mask == feat)
    print(tfm_time.segmentation_mask.data[locations_of_feature])
    # print(tfm_time.segmentation_mask[locations_of_feature].data)
    # print(locations_of_feature[0].shape, locations_of_feature[1].shape)
    break

In [ ]:
echo_top.plot()

In [ ]:
where18.any(dim='z').plot()

In [ ]:
(rdr.z.max() - )

In [ ]:
test_arr = np.array([False, False, True, True, True])
np.argmax(test_arr)

In [ ]:
x = np.array([[1, 0, 1], [0, 1, 0], [1, 0, 1]])
ones = np.where(x == 1)
only_the_ones = x[ones]
# for row, col in zip(*ones):
#     print(x[row, col])
print(only_the_ones)

In [ ]:
xr.open_dataset('/Volumes/LtgSSD/noaa-goes16/ABI-L2-MCMIPC/2022/152/00/OR_ABI-L2-MCMIPC-M6_G16_s20221520006173_e20221520008546_c20221520009050.nc')

In [ ]:
sat_data_test = xr.open_dataset('~/data/noaa-goes16/ABI-L2-ACHAC/2022/151/23/OR_ABI-L2-ACHAC-M6_G16_s20221512356173_e20221512358546_c20221520002383.nc')
sat_data_test.load()

In [ ]:
sat_temps = tfm.min_satellite_temp.data
min_min = np.nanmin(sat_temps)
print(min_min.compute())
max_max = np.nanmax(sat_temps)
print(max_max.compute())


In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.pcolormesh(sat_lon, sat_lat, feature_area_i_want.TEMP, transform=ccrs.PlateCarree())
ax.scatter(relevant_lons, relevant_lats, s=3, c='red', transform=ccrs.PlateCarree())
ax.scatter(sat_lon, sat_lat, s=3, c='gray', transform=ccrs.PlateCarree())
ax.add_feature(USCOUNTIES.with_scale('5m'))

In [ ]:
fig = plt.figure()
ax = fig.gca()
# ax.scatter(this_feature_x2d, this_feature_y2d, s=3, c='gray')
ax.scatter(grid_x2d[x_indices_valid, y_indices_valid], grid_y2d[x_indices_valid, y_indices_valid], s=3, c='r')
ax.pcolormesh(this_feature_x2d, this_feature_y2d, this_seg_mask, vmin=48, vmax=51, cmap='hot')
ax.pcolormesh(this_feature_x2d, this_feature_y2d, vals)


In [ ]:
def get_time(utc_nanoseconds):
    if type(utc_nanoseconds) == np.datetime64:
        return utc_nanoseconds
    else:
        return np.datetime64(utc_nanoseconds, 'ns')

In [ ]:
def get_time_str(utc_nanoseconds):
    time_str = dt.fromtimestamp(float(utc_nanoseconds) / 1e9, UTC).strftime('%Y-%m-%d %H:%M:%S')
    return time_str

In [ ]:
def plot_seg_mask(dataset, time):
    this_time = dataset.sel(time=time, method='nearest')
    seg_mask = this_time.segmentation_mask.data.copy().astype(float)
    seg_mask[seg_mask == 0] = np.nan
    plot = gv.QuadMesh((this_time.lon, this_time.lat, seg_mask), kdims=['lon', 'lat'], vdims=['segmentation mask']).opts(cmap='plasma', colorbar=False)
    return plot

In [ ]:
def plot_features(dataset, time):
    lats = dataset.feature_lat[dataset.feature_time == time]
    lons = dataset.feature_lon[dataset.feature_time == time]
    feat_ids = dataset.feature[dataset.feature_time == time]
    return gv.Points((lons, lats, feat_ids), kdims=['lon', 'lat'], vdims=['Feature ID']).opts(color='Cell ID', cmap='plasma', colorbar=True, tools=['hover'], size=4, line_color='k', line_width=0.5)

In [ ]:
col

In [ ]:
download_results

In [ ]:
l1b_ds_trimmed

In [ ]:
fk1 = 1.08033e+04
fk2 = 1.39274e+03
bc1 = 0.07550
bc2 = 0.99975

l1b_ds_trimmed['temp'] = (fk2 / (np.log((fk1 / l1b_ds_trimmed.Rad) + 1))  -  bc1) / bc2

In [ ]:
vmin = np.min([l1b_ds_trimmed.temp.min(), ctt_ds.TEMP.min()])
vmax = np.min([l1b_ds_trimmed.temp.max(), ctt_ds.TEMP.max()])

In [ ]:
geosys = coords.GeographicSystem()
satsys = coords.GeostationaryFixedGridSystem(subsat_lon=l1b_ds.nominal_satellite_subpoint_lon.data.item(), sweep_axis='x')

In [ ]:
x_2d, y_2d = np.meshgrid(l1b_ds_trimmed.x.data, l1b_ds_trimmed.y.data)

In [ ]:
lon, lat, alt = geosys.fromECEF(*satsys.toECEF(x_2d, y_2d, np.zeros_like(x_2d)))
lon = lon.reshape(x_2d.shape)
lat = lat.reshape(y_2d.shape)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
pcm = ax.pcolormesh(lon, lat, l1b_ds_trimmed.temp.data, vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
ax.add_feature(cfeat.COASTLINE)
ax.add_feature(cfeat.STATES)
fig.colorbar(pcm)

In [ ]:
ecef_coords = satsys.toECEF(ctt_ds.x.data, ctt_ds.y.data, np.zeros_like(ctt_ds.x.data))

In [ ]:
lon, lat, alt = geosys.fromECEF(*ecef_coords)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
pcm = ax.pcolormesh(lon, lat, ctt_ds.TEMP.data, vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
ax.add_feature(cfeat.COASTLINE)
ax.add_feature(cfeat.STATES)
fig.colorbar(pcm)

In [ ]:
for feat_id in tfm.feature:
    print(feat_id)
    this_time = tfm.isel(time=0)
    this_mask_lon = this_time.lon.where(this_time.segmentation_mask == feat_id)
    this_mask_lat = this_time.lat.where(this_time.segmentation_mask == feat_id)
    this_mask = this_time.segmentation_mask.where(this_time.segmentation_mask == feat_id)
    break

In [ ]:
this_mask_ecef = geosys.toECEF(this_time.lon, this_time.lat, np.zeros_like(this_time.lat))

In [ ]:
this_mask_x, this_mask_y, _ = satsys.fromECEF(*this_mask_ecef)

In [ ]:
fig = plt.figure()
ax = fig.gca()
pcm = ax.pcolormesh(l1b_ds_trimmed.x, l1b_ds_trimmed.y, l1b_ds_trimmed.temp, vmin=vmin, vmax=vmax)
mask = ax.pcolormesh(this_mask_x, this_mask_y, this_mask, cmap='Reds_r')
fig.colorbar(pcm)